In [1]:
# define the model
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.applications.inception_v3 import preprocess_input
from keras.models import Model, Sequential
from keras.layers import Input, Dense, concatenate, GlobalAveragePooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers.core import Dropout
from keras import backend as K
from keras import optimizers
from keras.utils import to_categorical

Using Theano backend.
WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: GeForce GTX 1060 6GB (CNMeM is disabled, cuDNN 5110)


In [2]:
img_feature_a = Input(shape=(2048,))
img_feature_b = Input(shape=(2048,))

shared_fc_layer = Sequential([
    Dense(1024, activation='relu', input_shape=(2048, )),
    BatchNormalization(),
    Dropout(0.5),
    Dense(256, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
])

encoded_a = shared_fc_layer(img_feature_a)
encoded_b = shared_fc_layer(img_feature_b)

merged_vector = concatenate([encoded_a, encoded_b])

x = Dense(256, activation='relu')(merged_vector)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
output = Dense(2, activation='softmax')(x)

rank_model = Model(inputs=[img_feature_a, img_feature_b], outputs=output)

rank_model.load_weights("./data/rank_model.h5")

In [3]:
base_model = InceptionV3(weights='imagenet', include_top=False)
x = base_model.output
x = GlobalAveragePooling2D()(x) # add a global spatial average pooling layer
inception_layer = Model(inputs=base_model.input, outputs=x)

img_a = Input(shape=(3, 299, 299))
img_b = Input(shape=(3, 299, 299))

img_feature_a = inception_layer(img_a)
img_feature_b = inception_layer(img_b)

output = rank_model([img_feature_a, img_feature_b])

model = Model(inputs=[img_a, img_b], outputs=output)

for layer in base_model.layers[:249]:
    layer.trainable = False
for layer in base_model.layers[249:]:
    layer.trainable = True
    

optimizer = optimizers.SGD(lr=0.0001, momentum=0.9)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_4 (InputLayer)             (None, 3, 299, 299)   0                                            
____________________________________________________________________________________________________
input_5 (InputLayer)             (None, 3, 299, 299)   0                                            
____________________________________________________________________________________________________
model_2 (Model)                  (None, 2048)          21802784    input_4[0][0]                    
                                                                   input_5[0][0]                    
____________________________________________________________________________________________________
model_1 (Model)                  (None, 2)             2498562     model_2[1][0]           

In [4]:
img_path = "../data/img/"
def img_preprocess(imgid, target_size=(299, 299)):
    try:
        filename = img_path+"%s.jpg"%imgid
        img = image.load_img(filename, target_size=target_size)
        x = image.img_to_array(img)
        x = preprocess_input(x)
        return x
    except Exception, e:
        print str(e)
        return None

In [5]:
from keras.preprocessing.image import *
import os

# 自定义DirectoryIterator类，可以返回自定义的label
class CustomDirectoryIterator(DirectoryIterator):  
    def next(self):
        """For python 2.x.
        # Returns
            The next batch.
        """
        with self.lock:
            index_array, current_index, current_batch_size = next(self.index_generator)
        # The transformation of images is not under thread lock
        # so it can be done in parallel
        batch_x1 = np.zeros((current_batch_size,) + self.image_shape, dtype=K.floatx())
        batch_x2 = np.zeros((current_batch_size,) + self.image_shape, dtype=K.floatx())
        batch_y = np.zeros((current_batch_size, 2) , dtype=K.floatx())
        # build batch of image data
        for i, j in enumerate(index_array):
            fname =  self.filenames[j]
            fname = os.path.basename(fname)
            fname, _ = os.path.splitext(fname)
            imgA, imgB, cmpret = fname.split("_")
            x1 = img_preprocess(imgA, self.target_size)
            x2 = img_preprocess(imgB, self.target_size)
            batch_x1[i] = x1
            batch_x2[i] = x2
            batch_y[i, int(cmpret)] = 1
        return [batch_x1, batch_x2], batch_y

# 定义批处理的数据集大小：较小的batch_size可以增加权重调整的次数，同时节省内存的开销
batch_size = 16 

# 图片预处理工具类
IDG = ImageDataGenerator()

# 从目录文件中流式读取数据，避免训练中一次性加载爆内存
train_batch = CustomDirectoryIterator("./data/finetune/train/", IDG, 
                                      target_size=(299, 299), batch_size=batch_size, shuffle=True)
valid_batch = CustomDirectoryIterator("./data/finetune/valid/", IDG, 
                                      target_size=(299, 299), batch_size=batch_size, shuffle=True)

Found 150716 images belonging to 1 classes.
Found 9216 images belonging to 1 classes.


In [ ]:
# 验证模型的初始化与rank model的结果相一致
import os
import cPickle as pickle

Y_predict_by_rank_model = pickle.load(open("./data/valid_predict_by_rank_model.pick", 'rb'))
Y_predict = model.predict_generator(valid_batch, steps=valid_batch.samples // batch_size + 1)

for i, fname in enumerate(valid_batch.filenames):
    fname = os.path.basename(fname)
    fname, _ = os.path.splitext(fname)
    imgA, imgB, _ = fname.split("_")
    if (imgA, imgB) not in Y_predict_by_rank_model:
        continue
    y_r = Y_predict_by_rank_model[(imgA, imgB)]
    y = Y_predict[i, 1]
    print y_r, y

In [ ]:
import lmdb

env = lmdb.open("./data/features")
txn = env.begin()

def testOutput(fname):
    fname = os.path.basename(fname)
    fname, _ = os.path.splitext(fname)
    imgA, imgB, cmpret = fname.split("_")
    
    rrr = model.predict([np.array([x1]), np.array([x2])])
    
    x1 = img_preprocess(imgA)
    x2 = img_preprocess(imgB)
    f1 = inception_layer.predict(np.array([x1]))
    f2 = inception_layer.predict(np.array([x2]))
    r = rank_model.predict([f1, f2])  
    
    fa = txn.get(imgA)
    fb = txn.get(imgB)
    fa = np.fromstring(fa, np.float32)
    fb = np.fromstring(fb, np.float32)
    rr = Y_predict_by_rank_model[(imgA, imgB)]
    return f1, f2, r, fa, fb, rr, rrr

testOutput(valid_batch.filenames[10])

In [ ]:
model.fit_generator(train_batch, steps_per_epoch=train_batch.samples // batch_size, epochs=10,
                       validation_data=valid_batch, validation_steps=valid_batch.samples // batch_size)

Epoch 1/10
9419/9419 [==============================] - 4733s - loss: 0.6601 - acc: 0.6233 - val_loss: 0.6731 - val_acc: 0.5913
Epoch 2/10
9419/9419 [==============================] - 4840s - loss: 0.6057 - acc: 0.6723 - val_loss: 0.6848 - val_acc: 0.5916
Epoch 3/10
9419/9419 [==============================] - 4840s - loss: 0.5584 - acc: 0.7170 - val_loss: 0.7084 - val_acc: 0.5919
Epoch 4/10
9419/9419 [==============================] - 4843s - loss: 0.5115 - acc: 0.7525 - val_loss: 0.7383 - val_acc: 0.5864
Epoch 5/10
9419/9419 [==============================] - 4839s - loss: 0.4644 - acc: 0.7844 - val_loss: 0.7591 - val_acc: 0.5882
Epoch 6/10
9419/9419 [==============================] - 4840s - loss: 0.4205 - acc: 0.8120 - val_loss: 0.7941 - val_acc: 0.5900
Epoch 7/10
9419/9419 [==============================] - 4839s - loss: 0.3809 - acc: 0.8330 - val_loss: 0.8079 - val_acc: 0.5856
Epoch 8/10
4273/9419 [============>.................] - ETA: 2528s - loss: 0.3509 - acc: 0.8501

In [ ]:
model.save_weights("./data/rank_model_ft.h5")

In [ ]:
# RUN ONLY ONCE
import cPickle as pickle
import random
import os
import shutil

train_list = pickle.load(open("./data/train.list", 'rb'))
valid_list = pickle.load(open("./data/valid.list", 'rb'))

if os.path.exists("./data/finetune"):
    shutil.rmtree("./data/finetune")
os.makedirs("./data/finetune/train/none")
os.makedirs("./data/finetune/valid/none")

white_set = set()
def touch_files(dataset, path, count=-1):
    if count != -1:
        subdata = random.sample(dataset, count)
    else:
        subdata = dataset
    for imgA, sA, imgB, sB, cmpret in subdata:
        if imgA in white_set or img_preprocess(imgA) is not None:
            white_set.add(imgA)
        if imgB in white_set or img_preprocess(imgB) is not None:
            white_set.add(imgB)
        if imgA in white_set and imgB in white_set:
            open(path+"/%s_%s_%d.jpg"%(imgA, imgB, cmpret), 'w').close()
        
touch_files(train_list, path="./data/finetune/train/none/", count=100000)
touch_files(valid_list, path="./data/finetune/valid/none/", count=5000)

In [ ]:
import os
import random
import shutil

path = "./data/finetune/valid/none/"
for fname in os.listdir(path):
    if random.random() < 0.8:
        os.remove(path+fname)